In [1]:
import os
import sys

spark_home = "/opt/apache-spark"
os.environ["SPARK_HOME"] = spark_home

# Add the PySpark directories to the Python path
sys.path.insert(0, os.path.join(spark_home, "python"))
sys.path.insert(0, os.path.join(spark_home, "python", "lib", "py4j-0.10.9-src.zip"))

In [2]:
import os
import sys
os.environ["PYSPARK_PYTHON"] = sys.executable
import pyspark 
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]") \
 .appName('SparkHelloWorld') \
 .getOrCreate()

ModuleNotFoundError: No module named 'py4j'

In [4]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
data = [
 ("James", "", "Smith", "36636", "M", 3000), 
 ("Michael", "Rose", "", "40288", "M", -1), 
 ("Robert", "", "Williams", "42114", "M", 4000), 
 ("Maria", "Anne", "Jones", "39192", "F", 4000), 
 ("Jen", "Mary", "Brown", None, "F", 3000)
]
schema = StructType([
 StructField("firstname", StringType(), True), \
 StructField("middlename", StringType(), True), \
 StructField("lastname", StringType(), True), \
 StructField("id", StringType(), True), \
 StructField("sex", StringType(), True), \
 StructField("salary", IntegerType(), True) \
 ])
df = spark.createDataFrame(data=data, schema=schema)
df.show()


+---------+----------+--------+-----+---+------+
|firstname|middlename|lastname|   id|sex|salary|
+---------+----------+--------+-----+---+------+
|    James|          |   Smith|36636|  M|  3000|
|  Michael|      Rose|        |40288|  M|    -1|
|   Robert|          |Williams|42114|  M|  4000|
|    Maria|      Anne|   Jones|39192|  F|  4000|
|      Jen|      Mary|   Brown| null|  F|  3000|
+---------+----------+--------+-----+---+------+



In [5]:
from pyspark.sql.functions import col
df_filtered = df.where(
 (col("id").isNotNull()) & (col("salary") > 0)
)
df_filtered.show()


+---------+----------+--------+-----+---+------+
|firstname|middlename|lastname|   id|sex|salary|
+---------+----------+--------+-----+---+------+
|    James|          |   Smith|36636|  M|  3000|
|   Robert|          |Williams|42114|  M|  4000|
|    Maria|      Anne|   Jones|39192|  F|  4000|
+---------+----------+--------+-----+---+------+



In [6]:
from pyspark.sql.functions import concat
df_full_name = df_filtered.withColumn(
 "fullname",
 concat(col("firstname"), col("middlename"), col("lastname"))
)
df_full_name.show()

+---------+----------+--------+-----+---+------+--------------+
|firstname|middlename|lastname|   id|sex|salary|      fullname|
+---------+----------+--------+-----+---+------+--------------+
|    James|          |   Smith|36636|  M|  3000|    JamesSmith|
|   Robert|          |Williams|42114|  M|  4000|RobertWilliams|
|    Maria|      Anne|   Jones|39192|  F|  4000|MariaAnneJones|
+---------+----------+--------+-----+---+------+--------------+



In [7]:
from pyspark.sql.functions import lit
df_full_name2 = df_filtered.withColumn(
 "fullname",
 concat(
 col("firstname"),
 lit(" "),
 col("middlename"),
 lit(" "),
 col("lastname"))
)
df_full_name2.show()


+---------+----------+--------+-----+---+------+----------------+
|firstname|middlename|lastname|   id|sex|salary|        fullname|
+---------+----------+--------+-----+---+------+----------------+
|    James|          |   Smith|36636|  M|  3000|    James  Smith|
|   Robert|          |Williams|42114|  M|  4000|Robert  Williams|
|    Maria|      Anne|   Jones|39192|  F|  4000|Maria Anne Jones|
+---------+----------+--------+-----+---+------+----------------+



In [8]:
df_filtered.createOrReplaceTempView("df")
df_full_name3 = spark.sql("""
 SELECT *,
 CASE
 WHEN middlename = '' THEN concat(firstname, " ", lastname)
 ELSE concat(firstname, " ", middlename, " ", lastname)
 END AS fullname
 FROM df
""")
df_full_name3.show()

+---------+----------+--------+-----+---+------+----------------+
|firstname|middlename|lastname|   id|sex|salary|        fullname|
+---------+----------+--------+-----+---+------+----------------+
|    James|          |   Smith|36636|  M|  3000|     James Smith|
|   Robert|          |Williams|42114|  M|  4000| Robert Williams|
|    Maria|      Anne|   Jones|39192|  F|  4000|Maria Anne Jones|
+---------+----------+--------+-----+---+------+----------------+



In [9]:
from pyspark.sql.functions import when
df_full_name4 = df_filtered.withColumn(
 "fullname",
 when(
 col("middlename") == "",
 concat(
 col("firstname"),
 lit(" "),
 col("lastname")
 )
 ).otherwise(
 concat(
 col("firstname"),
 lit(" "),
 col("lastname"),
 lit(" "),
 col("lastname")
 )
 )
)
df_full_name4.show()

+---------+----------+--------+-----+---+------+-----------------+
|firstname|middlename|lastname|   id|sex|salary|         fullname|
+---------+----------+--------+-----+---+------+-----------------+
|    James|          |   Smith|36636|  M|  3000|      James Smith|
|   Robert|          |Williams|42114|  M|  4000|  Robert Williams|
|    Maria|      Anne|   Jones|39192|  F|  4000|Maria Jones Jones|
+---------+----------+--------+-----+---+------+-----------------+

